# Configuring the extraction of efeatures

This notebook will demonstrate how to create the configuration files required for efeature extraction. Of course, it is a ficticious example as no ephys data is present in the current package.

The first step is to instantiate an access point to the local directory that will take care of storing the configuration:

In [1]:
emodel = "L5PC"

In [2]:
from bluepyemodel.access_point.local import LocalAccessPoint

access_point = LocalAccessPoint(
    emodel=emodel,
    final_path="./final.json",
    emodel_dir=f"./",
    recipes_path='./config/recipes.json',
)

Then, you will need to implement the function to gather the metadata of the files your are interested in as well as build a list of the targets:

In [5]:
import glob
import pathlib

protocols_rheobase = ["IDthresh"]

targets = [
    {
        "efeature": "Spikecount",
        "protocol": "IDRest",
        "amplitude": 200,
        "tolerance": 100,
        "efel_settings": {"strict_stiminterval": True}
    },
    {
        "efeature": "voltage_base",
        "protocol": "IDRest",
        "amplitude": 200,
        "tolerance": 100,
        "efel_settings": {"strict_stiminterval": True}
    },
    {
        "efeature": "AP_amplitude",
        "protocol": "IDRest",
        "amplitude": 200,
        "tolerance": 100,
        "efel_settings": {"strict_stiminterval": True}
    },  
    {
        "efeature": "AP_amplitude",
        "protocol": "APWaveform",
        "amplitude": 300,
        "tolerance": 100,
        "efel_settings": {"strict_stiminterval": True}
    }
]


def get_files_metadata():
    """ Build a list of files metadata of the form:

    [
        {
          "cell_name": "180124HuSHS1C3IN1",
          "filename": "180124HuSHS1C3IN1",
          "filepath": "./ephys_data/C010600C1-MT-C1.nwb",
          "ecodes": {
            "APWaveform": {}
          },
          "other_metadata": {
            "ljp": 14.0,
            "i_unit": "pA",
            "t_unit": "s",
            "v_unit": "mV"
          }
        },
        .
        .
        .
    ]
      """
    
    files_metadata = []
    
    ecodes = list(set([t["protocol"] for t in targets] + protocols_rheobase))

    for path in glob.glob("./ephys_data/*.nwb"):
        cell_name = pathlib.Path(path).stem
    
        files_metadata.append({
            "cell_name": cell_name,
            "filepath": path,
            "ecodes": {e: {} for e in ecodes}
        })

    return files_metadata

Then, we can create the configuration itself:

In [9]:
from bluepyemodel.efeatures_extraction.targets_configuration import TargetsConfiguration

config = TargetsConfiguration(
    files=get_files_metadata(),
    targets=targets,
    protocols_rheobase=protocols_rheobase,
)

Finally, it can be stored in the path indicated under the field "features" in the recipes:

In [7]:
access_point.store_targets_configuration(config)